In [1]:
import requests
from prettytable import PrettyTable
from typing import Dict, Optional, List

## hero winrate and counter

In [2]:
class FilterAPI:
    def __init__(self, url: str, payload: Dict):
        self.url = url
        self.payload = payload

    def fetch_data(self) -> Optional[Dict]:
        try:
            response = requests.post(self.url, json=self.payload)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            return None

class FilterFactory:
    URLS = {
        "1": "https://api.gms.moontontech.com/api/gms/source/2669606/2690817",
        "3": "https://api.gms.moontontech.com/api/gms/source/2669606/2690816",
        "7": "https://api.gms.moontontech.com/api/gms/source/2669606/2690519",
        "15": "https://api.gms.moontontech.com/api/gms/source/2669606/2674707",
        "30": "https://api.gms.moontontech.com/api/gms/source/2669606/2690818",
    }

    RANKS = {
        "glory": {"field": "bigrank", "operator": "eq", "value": "9"},
        "honor": {"field": "bigrank", "operator": "eq", "value": "8"},
        "mythic": {"field": "bigrank", "operator": "eq", "value": "7"},
        "legend": {"field": "bigrank", "operator": "eq", "value": "6"},
        "epic": {"field": "bigrank", "operator": "eq", "value": "5"},
        "all": {"field": "bigrank", "operator": "eq", "value": "101"},
    }

    @staticmethod
    def create_filter(day_filter: str, rank_filter: str, page_size: int, page_index: int, sorts_field: str, sorts_order: str) -> Optional[FilterAPI]:
        """
        Create a FilterAPI object based on the provided filter parameters.
        Args:
          day_filter: 1 day, 3 days, 7 days, 15 days, 30 days
          rank_filter: all, epic, legend, mythic, honor, glory
          page_size: 1 - 125
          page_index: 1 - 125
          sorts_field: main_hero_appearance_rate, main_hero_win_rate, main_hero_ban_rate
          sorts_order: asc, desc

        Returns:
          FilterAPI object or None if invalid filter type.
        """
        url = FilterFactory.URLS.get(day_filter)
        rank = FilterFactory.RANKS.get(rank_filter)

        if not url or not rank:
            print("Invalid day or rank filter.")
            return None

        payload = {
            "pageSize": page_size,
            "filters": [
                rank,
                {"field": "match_type", "operator": "eq", "value": "0"}
            ],
            "sorts": [
                {"data": {"field": sorts_field, "order": sorts_order}, "type": "sequence"},
                {"data": {"field": "main_heroid", "order": "desc"}, "type": "sequence"}
            ],
            "pageIndex": page_index,
            "fields": [
                "main_hero", "main_hero_appearance_rate", "main_hero_ban_rate",
                "main_hero_channel", "main_hero_win_rate", "main_heroid",
                "data.sub_hero.hero", "data.sub_hero.hero_channel",
                "data.sub_hero.increase_win_rate", "data.sub_hero.heroid"
            ]
        }

        return FilterAPI(url=url, payload=payload)

def create_hero_table(data: Dict, rate_filter: Optional[Dict[str, float]] = None) -> PrettyTable:
    table = PrettyTable()
    table.field_names = [
        "Hero ID",
        "Hero Name",
        # "Image URL",
        "Pick Rate",
        "Win Rate",
        "Ban Rate",
        "Counter Hero"
    ]
    table.align["Hero ID"] = "r"
    table.align["Hero Name"] = "l"
    table.align["Ban Rate"] = "r"
    table.right_padding_width = 1

    hero_name_dict = {record['data']['main_heroid']: record['data']['main_hero']['data']['name']
                      for record in data['data']['records']}

    for record in data['data']['records']:
        hero_data = record['data']['main_hero']['data']
        pick_rate = record['data']['main_hero_appearance_rate']
        win_rate = record['data']['main_hero_win_rate']
        ban_rate = record['data']['main_hero_ban_rate']

        # Apply the filter if specified
        if rate_filter:
            if "pick_rate" in rate_filter and pick_rate < rate_filter["pick_rate"]:
                continue
            if "win_rate" in rate_filter and win_rate < rate_filter["win_rate"]:
                continue
            if "ban_rate" in rate_filter and ban_rate < rate_filter["ban_rate"]:
                continue

        counter_hero_names = [hero_name_dict.get(hero['heroid'], "Unknown")
                              for hero in record['data']['sub_hero']]

        table.add_row([
            record['data']['main_heroid'],
            hero_data['name'],
            # hero_data['head'],
            f"{pick_rate:.2%}",
            f"{win_rate:.2%}",
            f"{ban_rate:.2%}",
            ','.join(counter_hero_names)
        ])

    return table


In [3]:
def main():
    day_filter               = "1" # 1 day, 3 days, 7 days, 15 days, 30 days
    rank_filter              = "glory" # all, epic, legend, mythic, honor, glory
    page_size                = 125
    page_index               = 1
    type_rate, rate          = "win_rate", 0
    sorts_field, sorts_order = "main_hero_win_rate", "desc"


    filter_factory = FilterFactory.create_filter(day_filter, rank_filter, page_size, page_index, sorts_field, sorts_order)

    if filter_factory:
        data = filter_factory.fetch_data()
        if data:
            table = create_hero_table(data, {type_rate: rate})
            print(table)
        else:
            print("Failed to retrieve data.")
    else:
        print("Invalid filter type.")

if __name__ == "__main__":
    main()

+---------+----------------+-----------+----------+----------+---------------------------------------------+
| Hero ID | Hero Name      | Pick Rate | Win Rate | Ban Rate |                 Counter Hero                |
+---------+----------------+-----------+----------+----------+---------------------------------------------+
|      20 | Lolita         |   0.67%   |  63.97%  |    8.79% |      Cyclops,Floryn,Bruno,Aamon,Granger     |
|      49 | Hylos          |   1.67%   |  57.12%  |   76.90% |      Eudora,Miya,Zilong,Gatotkaca,Layla     |
|      50 | Zhask          |   1.52%   |  56.45%  |   82.90% |       Harley,Wanwan,Selena,Arlott,Yin       |
|      48 | Diggie         |   0.51%   |  56.12%  |   12.61% |      Guinevere,Gloo,Cici,Alpha,Fredrinn     |
|     125 | Zhuxin         |   0.74%   |  56.02%  |   93.08% |       Baxia,Lolita,Hanabi,Alice,Martis      |
|     111 | Edith          |   1.31%   |  55.62%  |    2.12% |      Joy,Kadita,Fredrinn,Hilda,Cyclops      |
|      11 | Bane   

## hero best with, strong against, weak against

In [4]:
class HeroFilterAPI:
    def __init__(self, url: str, payload: Dict):
        self.url = url
        self.payload = payload

    def fetch_data(self) -> Optional[Dict]:
        try:
            response = requests.post(self.url, json=self.payload)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            return None

class HeroFilterFactory:
    @staticmethod
    def hero_create_filter(sortid_values: List[int], roadsort_values: List[int], page_size: int, page_index: int, sorts_field: str, sorts_order: str) -> FilterAPI:
        """
        Create a FilterAPI object based on the provided filter parameters.
        Args:
          sortid_values: List of integers representing sort IDs.
          roadsort_values: List of integers representing roadsort values.
          page_size: 1 - 125
          page_index: 1 - 125
          sorts_field: field name to sort by.
          sorts_order: asc, desc

        Returns:
          FilterAPI object.
        """
        url = "https://api.gms.moontontech.com/api/gms/source/2669606/2681577"

        payload = {
            "pageSize": page_size,
            "filters": [
                {"field": "sortid", "operator": "hasAnyOf", "value": sortid_values},
                {"field": "roadsort", "operator": "hasAnyOf", "value": roadsort_values}
            ],
            "sorts": [{"data": {"field": sorts_field, "order": sorts_order}, "type": "sequence"}],
            "pageIndex": page_index,
            "fields": ["roadsort", "channel", "sortid", "smallmap", "name", "heroid"]
        }

        return FilterAPI(url=url, payload=payload)

def hero_create_hero_table(data: Dict) -> PrettyTable:
    table = PrettyTable()
    table.field_names = [
        "Hero ID",
        "Hero Name",
        "Best With",
        "Strong Against",
        "Weak Against",
        "Desc Best With",
        "Desc Strong Against",
        "Desc Weak Against",
    ]
    table.align["Hero ID"] = "r"
    table.align["Hero Name"] = "l"

    table.align["Desc Best With"] = "l"
    table.align["Desc Strong Against"] = "l"
    table.align["Desc Weak Against"] = "l"
    table.right_padding_width = 1

    hero_name_dict = {
        125: "Zhuxin",
        124: "Chip",
        123: "Cici",
        122: "Nolan",
        121: "Ixia",
        120: "Arlott",
        119: "Novaria",
        118: "Joy",
        117: "Fredrinn",
        116: "Julian",
        115: "Xavier",
        114: "Melissa",
        113: "Yin",
        112: "Floryn",
        111: "Edith",
        110: "Valentina",
        109: "Aamon",
        108: "Aulus",
        107: "Natan",
        106: "Phoveus",
        105: "Beatrix",
        104: "Gloo",
        103: "Paquito",
        102: "Mathilda",
        101: "Yve",
        100: "Brody",
        99: "Barats",
        98: "Khaleed",
        97: "Benedetta",
        96: "Luo Yi",
        95: "Yu Zhong",
        94: "Popol and Kupa",
        93: "Atlas",
        92: "Carmilla",
        91: "Cecilion",
        90: "Silvanna",
        89: "Wanwan",
        88: "Masha",
        87: "Baxia",
        86: "Lylia",
        85: "Dyrroth",
        84: "Ling",
        83: "X.Borg",
        82: "Terizla",
        81: "Esmeralda",
        80: "Guinevere",
        79: "Granger",
        78: "Khufra",
        77: "Badang",
        76: "Faramis",
        75: "Kadita",
        74: "Minsitthar",
        73: "Harith",
        72: "Thamuz",
        71: "Kimmy",
        70: "Belerick",
        69: "Hanzo",
        68: "Lunox",
        67: "Leomord",
        66: "Vale",
        65: "Claude",
        64: "Aldous",
        63: "Selena",
        62: "Kaja",
        61: "Chang'e",
        60: "Hanabi",
        59: "Uranus",
        58: "Martis",
        57: "Valir",
        56: "Gusion",
        55: "Angela",
        54: "Jawhead",
        53: "Lesley",
        52: "Pharsa",
        51: "Helcurt",
        50: "Zhask",
        49: "Hylos",
        48: "Diggie",
        47: "Lancelot",
        46: "Odette",
        45: "Argus",
        44: "Grock",
        43: "Irithel",
        42: "Harley",
        41: "Gatotkaca",
        40: "Karrie",
        39: "Roger",
        38: "Vexana",
        37: "Lapu-Lapu",
        36: "Aurora",
        35: "Hilda",
        34: "Estes",
        33: "Cyclops",
        32: "Johnson",
        31: "Moskov",
        30: "Yi Sun-shin",
        29: "Ruby",
        28: "Alpha",
        27: "Sun",
        26: "Chou",
        25: "Kagura",
        24: "Natalia",
        23: "Gord",
        22: "Freya",
        21: "Hayabusa",
        20: "Lolita",
        19: "Minotaur",
        18: "Layla",
        17: "Fanny",
        16: "Zilong",
        15: "Eudora",
        14: "Rafaela",
        13: "Clint",
        12: "Bruno",
        11: "Bane",
        10: "Franco",
        9: "Akai",
        8: "Karina",
        7: "Alucard",
        6: "Tigreal",
        5: "Nana",
        4: "Alice",
        3: "Saber",
        2: "Balmond",
        1: "Miya"
    }

    # print(hero_name_dict)

    for record in data['data']['records']:
        hero_data = record['data']
        relation = hero_data['channel']['data']['relation']

        strong_against = ', '.join([hero_name_dict.get(hero_id, "Unknown") for hero_id in relation['strong']['target_hero_id']])
        weak_against = ', '.join([hero_name_dict.get(hero_id, "Unknown") for hero_id in relation['weak']['target_hero_id']])
        best_with = ', '.join([hero_name_dict.get(hero_id, "Unknown") for hero_id in relation['assist']['target_hero_id']])

        desc_strong_against = relation['strong']['desc']
        desc_weak_against = relation['weak']['desc']
        desc_best_with = relation['assist']['desc']

        table.add_row([
            hero_data['heroid'],
            hero_data['name'],
            best_with,
            strong_against,
            weak_against,
            desc_best_with,
            desc_strong_against,
            desc_weak_against,
        ])

    return table

In [5]:
def hero_main():
    sortid_values          = [1, 2, 3, 4, 5, 6]
    roadsort_values        = [1, 2, 3, 4, 5]
    page_size              = 125
    page_index             = 1
    sorts_field, sorts_order = "heroid", "asc"

    hero_filter_factory = HeroFilterFactory.hero_create_filter(sortid_values, roadsort_values, page_size, page_index, sorts_field, sorts_order)

    data = hero_filter_factory.fetch_data()
    if data:
        table = hero_create_hero_table(data)
        print(table)
    else:
        print("Failed to retrieve data.")

if __name__ == "__main__":
    hero_main()

+---------+----------------+------------------------------+-------------------------------+-------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Hero ID | Hero Name      |          Best With           | 

## hero detail

In [6]:
class HeroData:
    HERO_NAME_DICT = {
        125: "Zhuxin", 124: "Chip", 123: "Cici", 122: "Nolan", 121: "Ixia", 120: "Arlott", 119: "Novaria",
        118: "Joy", 117: "Fredrinn", 116: "Julian", 115: "Xavier", 114: "Melissa", 113: "Yin", 112: "Floryn",
        111: "Edith", 110: "Valentina", 109: "Aamon", 108: "Aulus", 107: "Natan", 106: "Phoveus", 105: "Beatrix",
        104: "Gloo", 103: "Paquito", 102: "Mathilda", 101: "Yve", 100: "Brody", 99: "Barats", 98: "Khaleed",
        97: "Benedetta", 96: "Luo Yi", 95: "Yu Zhong", 94: "Popol and Kupa", 93: "Atlas", 92: "Carmilla",
        91: "Cecilion", 90: "Silvanna", 89: "Wanwan", 88: "Masha", 87: "Baxia", 86: "Lylia", 85: "Dyrroth",
        84: "Ling", 83: "X.Borg", 82: "Terizla", 81: "Esmeralda", 80: "Guinevere", 79: "Granger", 78: "Khufra",
        77: "Badang", 76: "Faramis", 75: "Kadita", 74: "Minsitthar", 73: "Harith", 72: "Thamuz", 71: "Kimmy",
        70: "Belerick", 69: "Hanzo", 68: "Lunox", 67: "Leomord", 66: "Vale", 65: "Claude", 64: "Aldous",
        63: "Selena", 62: "Kaja", 61: "Chang'e", 60: "Hanabi", 59: "Uranus", 58: "Martis", 57: "Valir",
        56: "Gusion", 55: "Angela", 54: "Jawhead", 53: "Lesley", 52: "Pharsa", 51: "Helcurt", 50: "Zhask",
        49: "Hylos", 48: "Diggie", 47: "Lancelot", 46: "Odette", 45: "Argus", 44: "Grock", 43: "Irithel",
        42: "Harley", 41: "Gatotkaca", 40: "Karrie", 39: "Roger", 38: "Vexana", 37: "Lapu-Lapu", 36: "Aurora",
        35: "Hilda", 34: "Estes", 33: "Cyclops", 32: "Johnson", 31: "Moskov", 30: "Yi Sun-shin", 29: "Ruby",
        28: "Alpha", 27: "Sun", 26: "Chou", 25: "Kagura", 24: "Natalia", 23: "Gord", 22: "Freya", 21: "Hayabusa",
        20: "Lolita", 19: "Minotaur", 18: "Layla", 17: "Fanny", 16: "Zilong", 15: "Eudora", 14: "Rafaela",
        13: "Clint", 12: "Bruno", 11: "Bane", 10: "Franco", 9: "Akai", 8: "Karina", 7: "Alucard", 6: "Tigreal",
        5: "Nana", 4: "Alice", 3: "Saber", 2: "Balmond", 1: "Miya"
    }

    def __init__(self, url, payload):
        self.url = url
        self.payload = payload
        self.data = None

    def fetch_data(self):
        response = requests.post(self.url, json=self.payload)
        response.raise_for_status()
        self.data = response.json()

    def display_table(self, title, rows, field_names):
        table = PrettyTable()
        table.field_names = field_names
        for row in rows:
            table.add_row(row)
        print(f"\n{title}:")
        print(table)

    def display_main_hero(self):
        main_hero_data = self.data["data"]["records"][0]["data"]["main_hero"]["data"]
        main_hero = self.data['data']['records'][0]['data']
        main_hero_channel = self.data['data']['records'][0]['data']['main_hero_channel']

        rows = [
            # ["Image URL", main_hero_data["head"]],
            ["Hero ID", main_hero["main_heroid"]],
            ["Hero Name", main_hero_data["name"]],
            # ["Channel ID", main_hero_channel["id"]],
            # ["Match Type", main_hero["match_type"]],
            # ["Big Rank", main_hero["bigrank"]],
            # ["Camp Type", main_hero["camp_type"]],
            ["Pick Rate", f'{main_hero["main_hero_appearance_rate"] * 100:.2f}%'],
            ["Ban Rate", f'{main_hero["main_hero_ban_rate"] * 100:.2f}%'],
            ["Win Rate", f'{main_hero["main_hero_win_rate"] * 100:.2f}%'],
        ]
        self.display_table("Main Hero Information", rows, ["Attribute", "Value"])

    # counter relationship
    def display_sub_heroes_counter_relationship(self):
        sub_heroes = self.data["data"]["records"][0]["data"]["sub_hero"]
        rows = []
        for hero in sub_heroes:
            hero_name = self.HERO_NAME_DICT.get(hero["heroid"], "Unknown")
            row = [
                # hero["hero"]["data"]["head"],
                hero["hero_index"],
                # hero["hero_channel"]["id"],
                hero_name,
                f'{hero["increase_win_rate"] * 100:.2f}',
                f'{hero["hero_appearance_rate"] * 100:.2f}%',
                f'{hero["hero_win_rate"] * 100:.2f}%',
                f'{hero["min_win_rate10_12"] * 100:.2f}%',
                f'{hero["min_win_rate12_14"] * 100:.2f}%',
                f'{hero["min_win_rate14_16"] * 100:.2f}%',
                f'{hero["min_win_rate16_18"] * 100:.2f}%',
                f'{hero["min_win_rate18_20"] * 100:.2f}%',
                f'{hero["min_win_rate20"] * 100:.2f}%',
                f'{hero["min_win_rate6"] * 100:.2f}%',
                f'{hero["min_win_rate6_8"] * 100:.2f}%',
                f'{hero["min_win_rate8_10"] * 100:.2f}%'
            ]
            rows.append(row)
        self.display_table("Best Counters", rows, [
            # "Hero Image",
            "Index",
            # "Channel ID",
            "Hero Name",
            "Counter Score",
            "Pick Rate",
            "Win Rate",
            "Win Rate 10-12",
            "Win Rate 12-14",
            "Win Rate 14-16",
            "Win Rate 16-18",
            "Win Rate 18-20",
            "Win Rate 20+",
            "Win Rate 6+",
            "Win Rate 6-8",
            "Win Rate 8-10"
        ])

    def display_last_sub_heroes_counter_relationship(self):
        last_sub_heroes = self.data["data"]["records"][0]["data"]["sub_hero_last"]
        rows = []
        for hero in last_sub_heroes:
            hero_name = self.HERO_NAME_DICT.get(hero["heroid"], "Unknown")
            row = [
                hero["hero_index"],
                hero_name,
                f'{hero["increase_win_rate"] * 100:.2f}',
                f'{hero["hero_appearance_rate"] * 100:.2f}%',
                f'{hero["hero_win_rate"] * 100:.2f}%',
                f'{hero["min_win_rate10_12"] * 100:.2f}%',
                f'{hero["min_win_rate12_14"] * 100:.2f}%',
                f'{hero["min_win_rate14_16"] * 100:.2f}%',
                f'{hero["min_win_rate16_18"] * 100:.2f}%',
                f'{hero["min_win_rate18_20"] * 100:.2f}%',
                f'{hero["min_win_rate20"] * 100:.2f}%',
                f'{hero["min_win_rate6"] * 100:.2f}%',
                f'{hero["min_win_rate6_8"] * 100:.2f}%',
                f'{hero["min_win_rate8_10"] * 100:.2f}%'
            ]
            rows.append(row)
        self.display_table("Most Countered by", rows, [
            "Index",
            "Hero Name",
            "Counter Score",
            "Pick Rate",
            "Win Rate",
            "Win Rate 10-12",
            "Win Rate 12-14",
            "Win Rate 14-16",
            "Win Rate 16-18",
            "Win Rate 18-20",
            "Win Rate 20+",
            "Win Rate 6+",
            "Win Rate 6-8",
            "Win Rate 8-10"
        ])

    # compatibility
    def display_sub_heroes_compatibility(self):
        sub_heroes = self.data["data"]["records"][0]["data"]["sub_hero"]
        rows = []
        for hero in sub_heroes:
            hero_name = self.HERO_NAME_DICT.get(hero["heroid"], "Unknown")
            row = [
                # hero["hero"]["data"]["head"],
                hero["hero_index"],
                hero_name,
                f'{hero["increase_win_rate"] * 100:.2f}',
                f'{hero["hero_appearance_rate"] * 100:.2f}%',
                # hero["hero_channel"]["id"],
                f'{hero["hero_win_rate"] * 100:.2f}%',
                f'{hero["min_win_rate10_12"] * 100:.2f}%',
                f'{hero["min_win_rate12_14"] * 100:.2f}%',
                f'{hero["min_win_rate14_16"] * 100:.2f}%',
                f'{hero["min_win_rate16_18"] * 100:.2f}%',
                f'{hero["min_win_rate18_20"] * 100:.2f}%',
                f'{hero["min_win_rate20"] * 100:.2f}%',
                f'{hero["min_win_rate6"] * 100:.2f}%',
                f'{hero["min_win_rate6_8"] * 100:.2f}%',
                f'{hero["min_win_rate8_10"] * 100:.2f}%'
            ]
            rows.append(row)
        self.display_table("Compatibility", rows, [
            # "Hero Image",
            "Index",
            "Hero Name",
            "Teammate Score",
            "Pick Rate",
            # "Channel ID",
            "Win Rate",
            "Win Rate 10-12",
            "Win Rate 12-14",
            "Win Rate 14-16",
            "Win Rate 16-18",
            "Win Rate 18-20",
            "Win Rate 20+",
            "Win Rate 6+",
            "Win Rate 6-8",
            "Win Rate 8-10"
        ])

    def display_last_sub_heroes_compatibility(self):
        last_sub_heroes = self.data["data"]["records"][0]["data"]["sub_hero_last"]
        rows = []
        for hero in last_sub_heroes:
            hero_name = self.HERO_NAME_DICT.get(hero["heroid"], "Unknown")
            row = [
                hero["hero_index"],
                hero_name,
                f'{hero["increase_win_rate"] * 100:.2f}',
                f'{hero["hero_appearance_rate"] * 100:.2f}%',
                f'{hero["hero_win_rate"] * 100:.2f}%',
                f'{hero["min_win_rate10_12"] * 100:.2f}%',
                f'{hero["min_win_rate12_14"] * 100:.2f}%',
                f'{hero["min_win_rate14_16"] * 100:.2f}%',
                f'{hero["min_win_rate16_18"] * 100:.2f}%',
                f'{hero["min_win_rate18_20"] * 100:.2f}%',
                f'{hero["min_win_rate20"] * 100:.2f}%',
                f'{hero["min_win_rate6"] * 100:.2f}%',
                f'{hero["min_win_rate6_8"] * 100:.2f}%',
                f'{hero["min_win_rate8_10"] * 100:.2f}%'
            ]
            rows.append(row)
        self.display_table("Not Compatible", rows, [
            "Index",
            "Hero Name",
            "Teammate Score",
            "Pick Rate",
            "Win Rate",
            "Win Rate 10-12",
            "Win Rate 12-14",
            "Win Rate 14-16",
            "Win Rate 16-18",
            "Win Rate 18-20",
            "Win Rate 20+",
            "Win Rate 6+",
            "Win Rate 6-8",
            "Win Rate 8-10"
        ])

In [7]:
url = "https://api.gms.moontontech.com/api/gms/source/2669606/2690519"
payload = {
    "pageSize": 20,
    "filters": [
        {"field": "main_heroid", "operator": "eq", "value": 84},
        {"field": "bigrank", "operator": "eq", "value": "7"},
        {"field": "match_type", "operator": "eq", "value": "0"}
    ],
    "sorts": [],
    "pageIndex": 1
}

hero_data = HeroData(url, payload)

hero_data.fetch_data()

hero_data.display_main_hero()
hero_data.display_sub_heroes_counter_relationship()
hero_data.display_last_sub_heroes_counter_relationship()

# ============================================================

payload = {
    "pageSize": 20,
    "filters": [
        {"field": "main_heroid", "operator": "eq", "value": 84},
        {"field": "bigrank", "operator": "eq", "value": "7"},
        {"field": "match_type", "operator": "eq", "value": "1"}
    ],
    "sorts": [],
    "pageIndex": 1
}

hero_data = HeroData(url, payload)

hero_data.fetch_data()

hero_data.display_sub_heroes_compatibility()
hero_data.display_last_sub_heroes_compatibility()


Main Hero Information:
+-----------+--------+
| Attribute | Value  |
+-----------+--------+
|  Hero ID  |   84   |
| Hero Name |  Ling  |
| Pick Rate | 0.86%  |
|  Ban Rate | 76.52% |
|  Win Rate | 49.38% |
+-----------+--------+

Best Counters:
+-------+-----------+---------------+-----------+----------+----------------+----------------+----------------+----------------+----------------+--------------+-------------+--------------+---------------+
| Index | Hero Name | Counter Score | Pick Rate | Win Rate | Win Rate 10-12 | Win Rate 12-14 | Win Rate 14-16 | Win Rate 16-18 | Win Rate 18-20 | Win Rate 20+ | Win Rate 6+ | Win Rate 6-8 | Win Rate 8-10 |
+-------+-----------+---------------+-----------+----------+----------------+----------------+----------------+----------------+----------------+--------------+-------------+--------------+---------------+
|   1   |   Hanzo   |      4.95     |   0.28%   |  46.15%  |     59.06%     |     57.81%     |     58.09%     |     58.13%     |     57